In [15]:
import pandas as pd

# Load with correct delimiter (comma)
df = pd.read_csv("EQUITY_L.csv", sep=",", skipinitialspace=True)

# Clean headers
df.columns = df.columns.str.strip()

# Filter only active equity stocks
df = df[df['SERIES'] == 'EQ']

# Clean SYMBOL column
df['SYMBOL'] = df['SYMBOL'].str.strip()

# Get symbol list
symbols = df['SYMBOL'].tolist()
print("Total symbols to fetch:", len(symbols))


Total symbols to fetch: 1822


In [18]:
import pandas as pd
from nsepython import nse_eq
import time
from datetime import datetime
import os

# ========== Step 1: Load Symbol Master ==========
def load_symbols(csv_path):
    df = pd.read_csv(csv_path, sep=",", skipinitialspace=True)
    df.columns = df.columns.str.strip().str.replace('\ufeff', '', regex=True)
    df = df[df['SERIES'] == 'EQ']  # Keep only EQ
    df['SYMBOL'] = df['SYMBOL'].str.strip()
    return df['SYMBOL'].tolist()

# ========== Step 2: Fetch Price for One Symbol ==========
def get_nse_price(symbol, retries=3):
    for i in range(retries):
        try:
            data = nse_eq(symbol)
            return data['priceInfo']['lastPrice']
        except Exception as e:
            print(f"{symbol} attempt {i+1} failed: {e}")
            time.sleep(1)
    return None

# ========== Step 3: Process in Batches and Save ==========
def fetch_and_save(symbols, batch_size=100, output_dir="live_prices"):
    os.makedirs(output_dir, exist_ok=True)
    all_prices = []
    today = datetime.now().strftime("%Y-%m-%d")
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    for i in range(0, len(symbols), batch_size):
        batch = symbols[i:i+batch_size]
        print(f"Processing batch {i} to {i+len(batch)}")

        for symbol in batch:
            price = get_nse_price(symbol)
            all_prices.append({'SYMBOL': symbol, 'PRICE': price, 'FETCHED_AT': timestamp})
            print(f"{symbol}: ₹{price}")
            time.sleep(1)

        # Save after each batch
        batch_df = pd.DataFrame(all_prices)
        batch_df.to_csv(f"{output_dir}/live_prices_{today}.csv", index=False)

        print("Batch saved.")
        time.sleep(10)  # Pause between batches

    print(f"All {len(all_prices)} prices saved to {output_dir}/live_prices_{today}.csv")

In [7]:

# ========== Step 4: Run ==========
if __name__ == "__main__":
    symbols = load_symbols("EQUITY_L.csv")
    fetch_and_save(symbols)

Processing batch 0 to 100
20MICRONS: ₹265.8
21STCENMGM: ₹60.25
360ONE: ₹1071
3IINFOLTD: ₹23.66
3MINDIA: ₹31315
3PLAND: ₹46.9
5PAISA: ₹385.6
63MOONS: ₹1043
AAATECH: ₹87.56
AADHARHFC: ₹498
AAKASH: ₹9.9
AARON: ₹457
AARTECH: ₹68.3
AARTIDRUGS: ₹534.7
AARTIIND: ₹430.1
AARTIPHARM: ₹944
AARVI: ₹104.48
AAVAS: ₹1838.7
ABAN: ₹48.8
ABB: ₹5647
ABBOTINDIA: ₹33740
ABCAPITAL: ₹260.1
ABDL: ₹460.8
ABFRL: ₹73.77
ABMINTLLTD: ₹0
ABREL: ₹2008.1
ABSLAMC: ₹855
ACC: ₹1849
ACCELYA: ₹1402.5
ACCURACY: ₹7.6
ACE: ₹1105
ACEINTEG: ₹24.67
ACI: ₹666
ACL: ₹70.53
ACLGATI: ₹67
ACMESOLAR: ₹271.8
ACUTAAS: ₹1155
ADANIENSOL: ₹813.8
ADANIENT: ₹2549.5
ADANIGREEN: ₹978
ADANIPORTS: ₹1390
ADANIPOWER: ₹571
ADFFOODS: ₹267.9
ADOR: ₹1042
ADROITINFO: ₹10.81
ADSL: ₹173
ADVANIHOTR: ₹60.98
ADVENZYMES: ₹327
AEGISLOG: ₹734.85
AEGISVOPAK: ₹254.41
AEROFLEX: ₹205.28
AETHER: ₹781
AFCONS: ₹413.3
AFFLE: ₹1800
AFFORDABLE: ₹422
AFIL: ₹7.16
AFSL: ₹222.45
AGARIND: ₹943
AGARWALEYE: ₹456.1
AGI: ₹940.45
AGIIL: ₹1064
AGRITECH: ₹143
AGROPHOS: ₹40.6
AHLADA

In [22]:
import pandas as pd
import os

input_folder = "nse_historical_data"  # folder where your CSV files are
output_folder = "cleaned_stocks"

os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        filepath = os.path.join(input_folder, file)
        try:
            # Skip header lines with redundant stock names
            df = pd.read_csv(filepath, skiprows=2)

            # Case 1: 6 columns (Date, Close, High, Low, Open, Volume)
            if df.shape[1] == 6:
                df.columns = ['DATE', 'CLOSE', 'HIGH', 'LOW', 'OPEN', 'VOLUME']
                df['STOCK'] = file.replace('.csv', '') + '.NS'

            # Case 2: 7 columns (already includes stock name column)
            elif df.shape[1] == 7:
                df.columns = ['DATE', 'CLOSE', 'HIGH', 'LOW', 'OPEN', 'VOLUME', 'STOCK']

            else:
                print(f"Skipping {file}: unexpected number of columns ({df.shape[1]})")
                continue

            # Save cleaned file
            output_path = os.path.join(output_folder, file)
            df.to_csv(output_path, index=False)

        except Exception as e:
            print(f"Error processing {file}: {e}")


In [5]:
import os
import pandas as pd

# Set your folder path here
folder_path = "cleaned_stocks"  

# Empty list to store latest rows
latest_data = []

# Loop over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        try:
            # Read the CSV
            df = pd.read_csv(file_path)

            # Check if required columns exist
            if 'DATE' in df.columns:
                # Convert date to datetime
                df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')

                # Drop rows with invalid dates
                df = df.dropna(subset=['DATE'])

                # Sort and take latest row
                latest_row = df.sort_values('DATE').iloc[-1]

                # Add stock name from filename (remove .csv)
                latest_row['STOCK'] = filename.replace('.csv', '')

                # Append to list
                latest_data.append(latest_row)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Combine all latest rows
latest_df = pd.DataFrame(latest_data)

# Save to CSV
latest_df.to_csv("latest_stock_snapshot.csv", index=False)
print("Saved 'latest_stock_snapshot.csv' with latest row from each stock.")


Saved 'latest_stock_snapshot.csv' with latest row from each stock.


In [23]:
import pandas as pd
import os

all_data = []  # To hold enhanced data for each stock

folder_path = "cleaned_stocks"  # Folder with cleaned .csv files

for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        stock_name = file.replace('.csv', '')
        try:
            df = pd.read_csv(file_path)
            df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
            df = df.dropna(subset=['DATE'])
            df = df.sort_values('DATE')
            
            # Calculate daily return
            df['daily_return'] = df['CLOSE'].pct_change()
            
            # 30-day rolling metrics
            df['volatility_30d'] = df['daily_return'].rolling(30).std()
            df['avg_return_30d'] = df['daily_return'].rolling(30).mean()
            
            # Get latest available record (most recent row)
            latest = df.iloc[-1].copy()
            latest['STOCK'] = stock_name  # Add stock column
            all_data.append(latest)
        except Exception as e:
            print(f"Error processing {file}: {e}")

# Combine all
enhanced_df = pd.DataFrame(all_data)

# Categorize risk based on volatility
def get_risk(vol):
    if pd.isna(vol):
        return 'Unknown'
    if vol < 0.01:
        return 'Low'
    elif vol < 0.02:
        return 'Medium'
    else:
        return 'High'

enhanced_df['risk_level'] = enhanced_df['volatility_30d'].apply(get_risk)

# Save result
enhanced_df.to_csv("enhanced_stock_snapshot.csv", index=False)
print("Saved enhanced_stock_snapshot.csv with proper 30-day calculations.")



Saved enhanced_stock_snapshot.csv with proper 30-day calculations.


In [7]:
import os
import shutil

# Original file path (in current directory)
src_path = 'enhanced_stock_snapshot.csv'

# Destination folder
dst_folder = 'live_stocks_data_final'
os.makedirs(dst_folder, exist_ok=True)

# Full destination path
dst_path = os.path.join(dst_folder, 'enhanced_stock_snapshot.csv')

# Move file
shutil.move(src_path, dst_path)

print(f"File moved to: {dst_path}")


File moved to: live_stocks_data_final\enhanced_stock_snapshot.csv


In [8]:
import pandas as pd

# ------------------------------
# Load Enhanced Stock Snapshot
# ------------------------------
df = pd.read_csv("enhanced_stock_snapshot.csv")

# Rename for consistency
df.rename(columns={
    'avg_return_30d': 'avg_return',
    'ISIN': 'stock_id' if 'ISIN' in df.columns else 'STOCK'
}, inplace=True)

# If 'stock_id' doesn't exist, fallback to 'STOCK' column
if 'stock_id' not in df.columns:
    df['stock_id'] = df['STOCK']

# ------------------------------
# Simulated User Data
# ------------------------------
user_data = pd.DataFrame([
    {'user_id': 1, 'age': 28, 'gender': 'Male', 'income': 600000, 'risk_tolerance': 'High'},
    {'user_id': 2, 'age': 45, 'gender': 'Female', 'income': 800000, 'risk_tolerance': 'Medium'},
    {'user_id': 3, 'age': 35, 'gender': 'Male', 'income': 400000, 'risk_tolerance': 'Low'}
])

# ------------------------------
# Risk Level Mapping
# ------------------------------
risk_rank = {'Low': 1, 'Medium': 2, 'High': 3}

# ------------------------------
# Recommendation Logic
# ------------------------------
def recommend_stocks(user, stock_df, top_n=5):
    user_risk_rank = risk_rank[user['risk_tolerance']]

    # Filter stocks that match or are lower than user's risk tolerance
    eligible = stock_df[stock_df['risk_level'].map(risk_rank) <= user_risk_rank]

    # Drop rows with missing avg_return
    eligible = eligible.dropna(subset=['avg_return'])

    # Sort by highest avg return
    top_stocks = eligible.sort_values('avg_return', ascending=False).head(top_n)

    return top_stocks[['stock_id', 'avg_return', 'risk_level']]

# ------------------------------
# Run for Each User
# ------------------------------
for _, user in user_data.iterrows():
    print(f"\nRecommendations for User {user['user_id']} (Risk: {user['risk_tolerance']}):")
    print(recommend_stocks(user, df).to_string(index=False))



FileNotFoundError: [Errno 2] No such file or directory: 'enhanced_stock_snapshot.csv'

In [ ]:
all_recommendations = []

for _, user in user_data.iterrows():
    recs = recommend_stocks(user, df)
    recs['user_id'] = user['user_id']
    all_recommendations.append(recs)

final_df = pd.concat(all_recommendations, ignore_index=True)
final_df.to_csv("user_stock_recommendations.csv", index=False)
print("Saved recommendations to 'user_stock_recommendations.csv'")
